In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F # for activation functions. On the other hand, for adding act. func. we can use torch.nn as well.
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [5]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes): 
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 50) # 50 = hidden node counter
        self.fc2 = nn.Linear(50, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Test stage
model = NN(784, 10)
x = torch.randn(64, 784) # 64 row, 784 column tensor. REMINDER! = 784 is a shape of a single image size (28x28) in MNIST dataset.
print(model(x).shape)


torch.Size([64, 10])


In [10]:
# device setup

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [46]:
# hyperparameters

input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 10

In [47]:
# LOAD DATA

train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True) # ToTensor() converts original data that loaded from dataset library as np.array, to tensor. 
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True) # Shuffle makes us sure about do not have same images on batches on different epochs.

test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [48]:
# INIT MODEl
model = NN(input_size=input_size, num_classes=num_classes).to(device) # send to GPU if you have CUDA.  

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [49]:
# TRAINING

for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader): # data = img tensor, targets = labels tensor in related batch.
        data = data.to(device=device)
        targets = targets.to(device=device)

        # print(data.shape) # torch.Size([64, 1, 28, 28]) ==> 64 batch_size, 1 channel ( MNIST is a black-white image so has a 1 color channel)
                                                            #28 height,28 width

        data = data.reshape(data.shape[0], -1) # only important input i row number in this case. We hold only row number of torch size and flatten all the other dimensions. Every row holds a flattened image in this tensor. torch.Size([64, 784])

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad() # set gradient putput to zero for re-calculate gradient in every loop. We DO NOT want get interacted by previous gradient value.
        loss.backward() # update the weights

        # gradient descent or adam step 
        optimizer.step() # performs parameter update based on current gradient.

In [50]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking acc on training data")
    else:
        print("Checking acc on test data")

    num_correct = 0
    num_samples = 0
    model.eval() # switch to evaluating ( inference ) mode

    with torch.no_grad(): # this context manager guarantees us that we are NOT CALCULATING gradient during this process.
        for x,y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x)  # scores output shape = 64x10
            value, predictions = scores.max(1) # we take max value of second dimension for getting which number have been predicted by NN.
            num_correct += (predictions == y).sum()  
            num_samples += predictions.size(0)   
        print(f"Got{num_correct} / {num_samples} with acc, {float(num_correct)/float(num_samples)*100:.2f}") # 95.78 etc.
    
    model.train()
    
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking acc on training data
Got59056 / 60000 with acc, 98.43
Checking acc on test data
Got9697 / 10000 with acc, 96.97
